<a href="https://colab.research.google.com/github/nidhesg/Great-Frontend/blob/main/Copy_of_Model_Train_autocode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 1. Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# 2. Define correct path
file_path = "/content/drive/MyDrive/stackoverflow_qa.xlsx"

# 3. Read Excel file
import pandas as pd
df = pd.read_excel(file_path)   # requires openpyxl
print(df.head())


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
   QuestionId                                              Title  \
0      250001  Will the questions be migrated over from meta....   
1      250006                       Halp! The favicon is borked!   
2      250012              The system message theme is...jarring   
3      250013           The [featured] tag is not a mod-only tag   
4      250014  Why are we getting yearling badges for a site ...   

                                                Tags  \
0                    |discussion|meta|mso-mse-split|   
1                     |bug|status-completed|favicon|   
2  |feature-request|status-bydesign|design|system...   
3                        |bug|status-completed|tags|   
4        |bug|status-bydesign|badges|yearling-badge|   

                                        QuestionBody    QuestionCreationDate  \
0  Will we get our reputation moved over or q

In [ ]:
!ls /content/drive/MyDrive



'1589288763179_1589288550805_1589288548680_1589288534422_Nidhesh'\''s Resume (3).pdf'
 1604663865766.log
 adhr-01.PDF.gdoc
'Anyfile Notepad Files'
'ap (1).zip'
'ap (2).zip'
 ap.zip
'AXIS BANK  Statement for May 2020.pdf'
'beer consumption codes q.py'
 cil.pdf
 CoffeeBreakPythonSlicing.pdf
'Colab Notebooks'
'Copy of [FREE] Amazon FBA Fee Calculator.gsheet'
 demo.py
'Employment Application Form.docx'
'ezyzip (2).zip'
'Getting started.pdf'
 kkk
 manage.py
'Naukri_Nidheshresume(1).gdoc'
'Naukri_Nidheshresume(1).pdf'
'nidhesh-converted (1)-converted (1)-converted (2) (1).pdf'
'nidhesh-converted (1)-converted (1)-converted (2) (2).pdf'
'nidhesh-converted (1)-converted (1)-converted (2) (3).pdf'
'nidhesh-converted (1)-converted (1)-converted (2) (4).pdf'
'nidhesh-converted (1)-converted (1)-converted (2)_edited.pdf'
'nidhesh-converted (1)-converted (1)-converted (2).pdf'
'Nidhesh RuhelaNidhesh_InternshalaResume (1).pdf'
'Nidhesh Ruhela-  Python Backend developer- Pune- Solytics patner- Hird.d

In [ ]:
# If in Colab, mount Drive first (skip on Kaggle if using Kaggle Datasets):
IN_COLAB = True
try:
    from google.colab import drive  # noqa
    drive.mount('/content/drive')
except Exception:
    IN_COLAB = False
    pass

!pip -q install "transformers>=4.41" "datasets>=2.19" "accelerate>=0.33" peft openpyxl sentencepiece bitsandbytes


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 14.7 MB/s eta 0:00:00


In [ ]:
import os

# Path to your Excel file with Q/A
FILE_PATH = "/content/drive/MyDrive/stackoverflow_qa.xlsx"  # <- change if needed

# Columns in your Excel. Adjust if your sheet uses different names.
QUESTION_COL = "Question"
ANSWER_COL   = "Answer"

# Model + training
BASE_MODEL = "Salesforce/codet5p-220m"   # small & good for free GPUs
OUTPUT_DIR = "/content/codet5p_stackoverflow_lora"
MAX_SOURCE_LEN = 256
MAX_TARGET_LEN = 256
NUM_EPOCHS = 2
BATCH_SIZE = 4
LR = 2e-4
WARMUP_STEPS = 50

os.makedirs(OUTPUT_DIR, exist_ok=True)


In [ ]:
import pandas as pd
from datasets import Dataset, DatasetDict
import numpy as np

FILE_PATH = "/content/drive/MyDrive/stackoverflow_qa.xlsx"  # adjust if needed
df = pd.read_excel(FILE_PATH)

# Columns present:
# ['QuestionId','Title','Tags','QuestionBody', ..., 'Answer1_Body','Answer1_Score', ... up to 5]

# 1) Build QUESTION text = Title + two newlines + QuestionBody
def build_question(row):
    title = str(row.get("Title", "")).strip()
    body  = str(row.get("QuestionBody", "")).strip()
    if title and body:
        return f"{title}\n\n{body}"
    return (title or body)  # whichever exists

# 2) Pick the BEST answer body among Answer1..Answer5 by highest Score
answer_bodies = []
answer_scores = []
for i in range(1, 6):
    answer_bodies.append(df.get(f"Answer{i}_Body"))
    answer_scores.append(df.get(f"Answer{i}_Score"))

# Stack into arrays for easy argmax; replace NaNs in scores with a very low number
score_mat = np.column_stack([s if s is not None else np.full(len(df), -1e9) for s in answer_scores])
score_mat = np.where(np.isnan(score_mat), -1e9, score_mat)

# index of best answer per row
best_idx = score_mat.argmax(axis=1)

# gather best bodies
body_mat = np.column_stack([b if b is not None else np.array([""]*len(df)) for b in answer_bodies])
best_answer = [body_mat[r, best_idx[r]] if best_idx[r] >= 0 else "" for r in range(len(df))]

# 3) Assemble minimal Q/A frame and clean
qa_df = pd.DataFrame({
    "question": [build_question(r) for _, r in df.iterrows()],
    "answer": best_answer
})

# Remove rows with missing/empty Q or A
qa_df["question"] = qa_df["question"].fillna("").astype(str).str.strip()
qa_df["answer"]   = qa_df["answer"].fillna("").astype(str).str.strip()
qa_df = qa_df[(qa_df["question"] != "") & (qa_df["answer"] != "")].reset_index(drop=True)

print("Sample rows:")
print(qa_df.head(3))

# 4) Split train/validation
split = int(len(qa_df) * 0.9) if len(qa_df) > 10 else max(1, len(qa_df) - 1)
train_df = qa_df.iloc[:split].reset_index(drop=True)
val_df   = qa_df.iloc[split:].reset_index(drop=True)

ds = DatasetDict({
    "train": Dataset.from_pandas(train_df),
    "validation": Dataset.from_pandas(val_df)
})
ds
